In [1]:
from tensorflow.keras.datasets import mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

/Users/weikai/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/weikai/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/weikai/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/weikai/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework

In [2]:
import numpy as np
np.random.seed(1)

def one_hot(x:list):
    return np.eye(10)[x]

In [3]:
X_train = X_train.reshape(60000, 784)
Y_train = one_hot(Y_train)

In [4]:
X_train = X_train / 255.0

In [53]:
def sigmoid(x, d=False):
    if d:
        return sigmoid(x) * (1 - sigmoid(x))
    return 1 / (1 + np.exp(-x))

# configs
iteration = 100000
learning_rate = 0.01
data_size = 60000
batch_size = 16
input_size = 784
output_size = 10
hidden_1 = 128
hidden_2 = 32

# weights init
weights_0 = np.random.uniform(-1, 1, (input_size, hidden_1))
weights_1 = np.random.uniform(-1, 1, (hidden_1, hidden_2))
weights_2 = np.random.uniform(-1, 1, (hidden_2, output_size))

# bias init
b_0 = 1
b_1 = 1
b_2 = 1

In [54]:
from tqdm import trange

t = trange(iteration)

# training network
for i in t:
    # batch
    a = (i % int(data_size / batch_size - 1)) * batch_size
    b = ((i % int(data_size / batch_size - 1)) + 1) * batch_size
    tmp_X_train = X_train[a:b]
    tmp_Y_train = Y_train[a:b]
    # forward
    p1 = np.dot(tmp_X_train, weights_0) + b_0
    act1 = sigmoid(p1)
    p2 = np.dot(act1, weights_1) + b_1
    act2 = sigmoid(p2)
    p3 = np.dot(act2, weights_2) + b_2
    act3 = sigmoid(p3)
    err = (tmp_Y_train - act3) ** 2
    m_err = np.mean(np.sum(err, axis=0))
    t.set_description("mean error: %f" % m_err)
    # backward
    t1 = 2 * (tmp_Y_train - act3) * sigmoid(p3, True)
    t2 = np.dot(act2.T, t1)
    t_1 = np.dot(weights_2, t1.T)
    t_2 = sigmoid(p2, True) * t_1.T
    t_3 = np.dot(act1.T, t_2)
    k1 = np.dot(weights_1, t_2.T)
    k2 = sigmoid(p1, True) * k1.T
    k3 = np.dot(tmp_X_train.T, k2)
    weights_0 = weights_0 + (learning_rate * k3)
    weights_1 = weights_1 + (learning_rate * t_3)
    weights_2 = weights_2 + (learning_rate * t2)

mean error: 0.000633: 100%|██████████| 100000/100000 [08:17<00:00, 200.90it/s]


In [76]:
counter = 0

it = 10000
t = trange(it)

for i in t:
    test = X_test[i].flatten()
    p1 = np.dot(test, weights_0) + b_0
    act1 = sigmoid(p1)
    p2 = np.dot(act1, weights_1) + b_1
    act2 = sigmoid(p2)
    p3 = np.dot(act2, weights_2) + b_2
    act3 = sigmoid(p3)
    pred = np.argmax(act3)
    ans = Y_test[i]
    if pred == ans:
        counter = counter + 1
        t.set_description("accuracy: %d / %d" % (counter, it))

  0%|          | 0/10000 [00:00<?, ?it/s]/Users/weikai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
accuracy: 9456 / 10000: 100%|██████████| 10000/10000 [00:36<00:00, 270.35it/s]
